In [1]:
# OG DATASET --- Build WIDE Energy table: ObjectId; Country; Continent; <TechUnit>__FYYYY for all years ---

import re
import pandas as pd

# 1) Load + ISO2 filter
csv_path = "Renewable_Energy.csv"
f = pd.read_csv(csv_path)
iso2_clean = f["ISO2"].astype(str).str.strip()
f = f[iso2_clean.ne("") & f["ISO2"].notna()].copy()

# 2) Detect FYYYY year columns
year_cols = sorted([c for c in f.columns if re.fullmatch(r"F\d{4}", c)])
for c in year_cols:
    f[c] = pd.to_numeric(f[c], errors="coerce")

# 3) Standardize Technology + extract MW/GWh
tech_map = {
    "Hydropower (excl. Pumped Storage)": "Hydro",
    "Solar energy": "Solar",
    "Wind energy": "Wind",
    "Bioenergy": "Bio",
    "Fossil fuels": "Fossil",
}
f["Technology_std"] = f["Technology"].map(tech_map).fillna(f["Technology"])

def extract_abbr(unit: str):
    if not isinstance(unit, str):
        return None
    m = re.search(r"\b(MW|GWh)\b", unit, flags=re.IGNORECASE)
    return m.group(1).upper() if m else None

f["UnitAbbr"] = f["Unit"].apply(extract_abbr)
f = f[f["UnitAbbr"].notna()].copy()

# 4) TechUnit = "<Tech> (<MW|GWh>)"
f["TechUnit"] = f["Technology_std"].astype(str).str.strip() + " (" + f["UnitAbbr"] + ")"

# 5) Continent from ISO2 using libraries (comprehensive)

def add_continent_from_iso2(df: pd.DataFrame) -> pd.DataFrame:
    s = df["ISO2"].astype(str).str.strip().str.upper().replace({"": None, "NA": None, "NAN": None})

    # Preferred: country_converter
    try:
        import country_converter as coco
        cc = coco.CountryConverter()
        # coco returns a list; convert back to a Series aligned to index
        cont_list = cc.convert(names=s.tolist(), src="ISO2", to="continent", not_found=None)
        cont_series = pd.Series(cont_list, index=df.index)
        df["Continent"] = cont_series.fillna("Unknown")
        return df
    except Exception:
        pass

    # Fallback: pycountry_convert
    try:
        from pycountry_convert import (
            country_alpha2_to_continent_code as a2_to_cc,
            convert_continent_code_to_continent_name as cc_to_name,
        )
        def conv(a2):
            try:
                return cc_to_name(a2_to_cc(a2))
            except Exception:
                return None
        cont_series = s.map(conv).fillna("Unknown")
        df["Continent"] = cont_series
        return df
    except Exception:
        # Last resort: mark unknown (so pipeline doesn’t break)
        df["Continent"] = "Unknown"
        return df

# Apply
f = add_continent_from_iso2(f)

# 6) Ensure ObjectId exists (if not in CSV, synthesize)
if "ObjectId" not in f.columns:
    f["ObjectId"] = range(1, len(f) + 1)

# 7) Build WIDE table: index = (ObjectId, Country, Continent); columns = "<TechUnit>__FYYYY"
#   a) aggregate per country+techunit (sum over duplicates)
agg = f.groupby(["Country","Continent","TechUnit"], as_index=False)[year_cols].sum()    #! DEBUG REMOVED: "ObjectId",
#   b) long -> add combined column name "<TechUnit>__FYYYY"
long = agg.melt(id_vars=["Country","Continent","TechUnit"],                             #! DEBUG REMOVED: "ObjectId",
                value_vars=year_cols, var_name="YearCol", value_name="Value")
long["col"] = long["TechUnit"] + "__" + long["YearCol"]
#   c) wide pivot
wide = (long.pivot_table(index=["Country","Continent"], columns="col",                  #! DEBUG REMOVED: "ObjectId",
                         values="Value", aggfunc="sum")
            .reset_index()
            .fillna(0.0))

# 8) Output objects you’ll use later
df_wide = wide  # columns: ObjectId, Country, Continent, <TechUnit>__FYYYY, ...
tech_options = sorted(f["TechUnit"].unique().tolist())  # all TechUnits (MW+GWh)
xy_kwargs = {f"XY_var{i+1}": t for i, t in enumerate(tech_options)}  # for widget call

print(f"Wide shape: {df_wide.shape}  | TechUnits: {len(tech_options)}  | Years: {len(year_cols)}")

# Consistent color palette for continents
continent_colors = {
    "Asia": "#1f77b4",
    "Europe": "#ff7f0e",
    "Africa": "#2ca02c",
    "Oceania": "#d62728",
    "America": "#9467bd",
}

df_wide

Wide shape: (219, 232)  | TechUnits: 10  | Years: 23


col,Country,Continent,Bio (GWH)__F2000,Bio (GWH)__F2001,Bio (GWH)__F2002,Bio (GWH)__F2003,Bio (GWH)__F2004,Bio (GWH)__F2005,Bio (GWH)__F2006,Bio (GWH)__F2007,...,Wind (MW)__F2013,Wind (MW)__F2014,Wind (MW)__F2015,Wind (MW)__F2016,Wind (MW)__F2017,Wind (MW)__F2018,Wind (MW)__F2019,Wind (MW)__F2020,Wind (MW)__F2021,Wind (MW)__F2022
0,"Afghanistan, Islamic Rep. of",Asia,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.1,0.1,0.1,0.1,0.4,0.4,0.40,0.4,0.4,0.4
1,Albania,Europe,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
2,Algeria,Africa,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.0,10.2,10.2,10.2,10.2,10.0,10.00,10.0,10.0,10.0
3,American Samoa,Oceania,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
4,"Andorra, Principality of",Europe,0.000,0.000,0.000,0.000,1.000,1.000,4.000,19.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,Vietnam,Asia,300.886,293.138,342.400,337.094,312.986,298.974,334.390,347.934,...,53.0,53.0,136.0,160.2,204.7,236.7,374.55,518.0,4118.0,5065.0
215,West Bank and Gaza,Asia,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
216,"Yemen, Rep. of",Asia,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
217,Zambia,Africa,39.013,39.013,39.013,39.013,39.013,39.366,39.013,38.660,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0


In [2]:
from Isea.scatter import ScatterBrush
import re

# Year range
years_int = sorted({int(m.group(1)) for c in df_wide.columns
                    for m in [re.search(r"__F(\d{4})$", str(c))] if m})
year_min, year_max = years_int[0], years_int[-1]


xy_kwargs = {f"XY_var{i+1}": t for i, t in enumerate(tech_options, start=1)}

rows = df_wide.to_dict("records")   # full table; widget remaps per year

w_scatter = ScatterBrush(
    data=rows,
    x=tech_options[1], y=tech_options[3],
    color="Continent",
    colorMap=continent_colors,
    label="Country",      # ✅ display name
    key="Country",        # ✅ stable unique id (one row per country now)
    width=1200, height=500,
    panel_position="right", panel_width=320, panel_height=220,
    YearMin=year_min, YearMax=year_max,
    **xy_kwargs,
)
display(w_scatter)

ScatterBrush(data=[{'Country': 'Afghanistan, Islamic Rep. of', 'Continent': 'Asia', 'Bio (GWH)__F2000': 0.0, '…

In [3]:
# --- Minimal linked view: second scatter shows only the selected countries ---

import pandas as pd
from Isea.scatter import ScatterBrush

# Pull current axis/fields from the first widget (robust to either attr or .options)
opts = getattr(w_scatter, "options", {}) or {}
x_col = getattr(w_scatter, "x", None) or opts.get("x")
y_col = getattr(w_scatter, "y", None) or opts.get("y")
label_col = getattr(w_scatter, "label", None) or opts.get("label") or "Country"
color_col = getattr(w_scatter, "color", None) or opts.get("color") or "Continent"
key_col   = getattr(w_scatter, "key", None)   or opts.get("key")   or "Country"


# print(opts, x_col, y_col, label_col, color_col, key_col)
# XY_var* and year range from earlier cells
xy_kwargs = {f"XY_var{i+1}": t for i, t in enumerate(tech_options, start=1)}

# Empty second widget (data will be filled on first selection)
w_scatter_sel = ScatterBrush(
    data=pd.DataFrame([], columns=[label_col, color_col, key_col]).to_dict("records"),
    x=x_col, y=y_col,
    color=color_col,colorMap=continent_colors, 
    label=label_col, key=key_col,
    width=900, height=450, panel_position="right", panel_height=160,
    # title="Linked view (selection from first scatter)",
    YearMin=year_min, YearMax=year_max,
    **xy_kwargs,
)
display(w_scatter_sel)

# Keep a df for downstream cells
df_selected = pd.DataFrame(columns=df_wide.columns)


def _link_selection_to_second(change):
    global df_selected
    sel = change.get("new") or {}
    rows = sel.get("rows") or []

    # countries from the selection (row has Country or fallback to label/key)
    countries = [r.get("Country") or r.get(label_col) or r.get(key_col) for r in rows]
    countries = [c for c in countries if isinstance(c, str)]

    # Update df_selected (full wide rows so year/XY remain interactive)
    if countries:
        df_selected = df_wide[df_wide["Country"].isin(countries)].copy()
    else:
        df_selected = pd.DataFrame(columns=df_wide.columns)

    # Keep linked widget in sync:
    #  - data: full rows (so its slider & XY pills work)
    #  - axes: mirror current axes from the first widget
    cur_x = getattr(w_scatter, "x", None) or opts.get("x")
    cur_y = getattr(w_scatter, "y", None) or opts.get("y")
    w_scatter_sel.data = (
        df_selected.assign(key=lambda d: d["Country"], label=lambda d: d["Country"])
                   .to_dict("records")
    )
    if cur_x and cur_y:
        w_scatter_sel.x = cur_x
        w_scatter_sel.y = cur_y

    # optional: clear selection inside the second chart each update
    w_scatter_sel.selection = {"type": None, "keys": [], "rows": [], "epoch": int(__import__("time").time()*1000)}

# Wire first -> second (live updates)
w_scatter.observe(_link_selection_to_second, names="selection")

C:\Users\Usuario\AppData\Local\Temp\ipykernel_22168\3331563500.py:21: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  data=pd.DataFrame([], columns=[label_col, color_col, key_col]).to_dict("records"),


ScatterBrush(options={'x': 'Bio (MW)', 'y': 'Fossil (MW)', 'label': 'Country', 'color': 'Continent', 'key': 'C…

In [4]:
# ---- Minimal EV scatter prep (Region×Mode; exact column names) ----
import pandas as pd
import re
from Isea.scatter import ScatterBrush

df = pd.read_csv("Global_EV_clean.csv")

# Helper: pivot (region,mode,year) → (region,mode) with <name>__FYYYY columns
def pivot_param(data, parameter, powertrain=None, name="Val"):
    q = data[data["parameter"] == parameter]
    if powertrain is not None:
        q = q[q["powertrain"] == powertrain]
    q = q[q["mode"] != "EV"]                                 # drop generic mode
    if q.empty:
        return pd.DataFrame(columns=["region","mode"])
    g = q.groupby(["region","mode","year"], as_index=False)["value"].sum()
    p = g.pivot(index=["region","mode"], columns="year", values="value").sort_index(axis=1)
    p.columns = [f"{name}__F{int(y)}" for y in p.columns]
    return p.reset_index()

# ChargingStations: sum fast+slow per region-year, replicate to all non-EV modes, then pivot
cp = df[(df["parameter"] == "EV charging points") & (df["powertrain"].isin(["fast","slow"]))]
rgy = (cp.groupby(["region","year"], as_index=False)["value"].sum()
         .rename(columns={"value":"ChargingStations"}))
rgy["year"] = rgy["year"].astype(int)
base_modes = df.loc[df["mode"] != "EV", ["region","mode"]].drop_duplicates()
rep = base_modes.merge(rgy, on="region", how="left")
cs = (rep.pivot_table(index=["region","mode"], columns="year", values="ChargingStations", aggfunc="first")
         .sort_index(axis=1)
         .reset_index())
cs.columns = [f"ChargingStations__F{int(c)}" if c not in ("region","mode") else c for c in cs.columns]

# Metrics for X/Y
cfg = {
    "StockBEV":   ("EV stock",       "BEV"),
    "StockFCEV":  ("EV stock",       "FCEV"),
    "StockPHEV":  ("EV stock",       "PHEV"),
    "SalesBEV":   ("EV sales",       "BEV"),
    "SalesFCEV":  ("EV sales",       "FCEV"),
    "SalesPHEV":  ("EV sales",       "PHEV"),
    "SalesShare": ("EV sales share", "EV"),
    "StockShare": ("EV stock share", "EV"),
}
blocks = [pivot_param(df, p, pt, name=k) for k,(p,pt) in cfg.items()] + [cs]

# Merge on (region, mode)
wide = base_modes.copy()
for b in blocks:
    if not b.empty:
        wide = wide.merge(b, on=["region","mode"], how="outer")

# Year range + initialize bare columns for latest year (widget expects this)
yrs = sorted({int(m.group(1)) for c in wide.columns for m in [re.search(r"__F(\d{4})$", str(c))] if m})
yearMin, yearMax = (min(yrs), max(yrs)) if yrs else (None, None)

xyVars = list(cfg.keys()) + ["ChargingStations"]
if yearMax is not None:
    for v in xyVars:
        col = f"{v}__F{yearMax}"
        wide[v] = pd.to_numeric(wide[col], errors="coerce").fillna(0.0) if col in wide.columns else 0.0

# Minimal id/label; color by mode
wide["id"] = wide["region"] + "|" + wide["mode"]
wide["label"] = wide["region"] + " • " + wide["mode"]

# Scatter
w = ScatterBrush(
    wide,
    x="ChargingStations",   # defaults; switch in UI
    y="SalesBEV",
    key="id",
    label="label",
    color="mode",
    legend=True,
    legendPosition="right",
    xyVars=xyVars,
    yearMin=yearMin,
    yearMax=yearMax,
    width=1080,
    height=520,
    panel_position="right",
    panel_width=340,
)
w


ScatterBrush(data=[{'region': 'Australia', 'mode': 'Cars', 'StockBEV__F2010': None, 'StockBEV__F2011': 49.0, '…

In [5]:
# --- Minimal linked view: second scatter mirrors selection + axes (options-based) ---
from Isea.scatter import ScatterBrush

def _get_axes(widget):
    opts = getattr(widget, "options", {}) or {}
    return opts.get("x"), opts.get("y")

x0, y0 = _get_axes(w)

# empty second widget; same config as the first
w_link = ScatterBrush(
    wide.iloc[0:0].to_dict("records"),
    x=x0, y=y0,
    key="id", label="label", color="mode",
    legend=True, legendPosition="right",
    xyVars=xyVars, yearMin=yearMin, yearMax=yearMax,
    width=1080, height=420, panel_position="right", panel_width=340,
)
display(w_link)

def _sync(*_):
    # current axes from the first widget
    x, y = _get_axes(w)
    # current selection
    sel = w.selection if isinstance(w.selection, dict) else {}
    keys = sel.get("keys", []) or []
    sub = wide[wide["id"].isin(keys)] if keys else wide.iloc[0:0]

    # push data
    w_link.data = sub.to_dict("records")
    # push axes via options (keep other options intact)
    w_link.options = {**(w_link.options or {}), "x": x, "y": y}
    # optional: clear selection inside the second chart each update
    w_link.selection = {"type": None, "keys": [], "rows": [], "epoch": 0}

# react to selection and option (axis) changes
w.observe(lambda ch: _sync(), names="selection")
w.observe(lambda ch: _sync(), names="options")

# initial paint
_sync()


ScatterBrush(options={'x': 'ChargingStations', 'y': 'SalesBEV', 'label': 'label', 'color': 'mode', 'key': 'id'…

In [6]:
# ===== CELL: Clustering Analysis =====
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# 1) CLUSTERING: Group countries by renewable energy profile (latest year)
def cluster_countries_by_energy_profile(df_wide, n_clusters=4):
    """
    Cluster countries based on their renewable energy mix.
    Returns: DataFrame with cluster assignments
    """
    # Extract latest year columns for each technology
    tech_cols = [c for c in df_wide.columns if '__F' in c]
    latest_year = max([int(c.split('__F')[1]) for c in tech_cols])
    
    # Get columns for latest year
    latest_cols = [c for c in tech_cols if f'__F{latest_year}' in c]
    
    # Prepare data for clustering
    X = df_wide[['Country', 'Continent'] + latest_cols].copy()
    X_numeric = X[latest_cols].fillna(0)
    
    # Standardize features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_numeric)
    
    # K-means clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    X['Cluster'] = kmeans.fit_predict(X_scaled)
    
    # Add cluster statistics
    cluster_stats = X.groupby('Cluster')[latest_cols].mean()
    
    return X[['Country', 'Continent', 'Cluster']], cluster_stats

clusters_df, cluster_stats = cluster_countries_by_energy_profile(df_wide, n_clusters=4)
print("Countries by cluster:")
print(clusters_df.groupby('Cluster')['Country'].apply(list))
print("\nCluster characteristics (mean values):")
print(cluster_stats)

Countries by cluster:
Cluster
0    [Afghanistan, Islamic Rep. of, Albania, Algeri...
1                              [China, P.R.: Mainland]
2      [Brazil, Germany, India, Japan, United Kingdom]
3                                      [United States]
Name: Country, dtype: object

Cluster characteristics (mean values):
col      Bio (GWH)__F2022  Bio (MW)__F2022  Fossil (GWH)__F2022  \
Cluster                                                           
0             1092.311429       253.705094         3.147871e+04   
1           129700.000000     29390.000000         5.705850e+06   
2            40099.915200     10001.678600         5.033481e+05   
3            57146.000000     11101.500000         2.696236e+06   

col      Fossil (MW)__F2022  Hydro (GWH)__F2022  Hydro (MW)__F2022  \
Cluster                                                              
0              8.355291e+03        9.811817e+03        2889.256255   
1              1.295650e+06        1.303900e+06      367710.000000  

c:\Users\Usuario\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


In [7]:

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import IsolationForest
from sklearn.metrics import r2_score
import seaborn as sns
from IPython.display import display, HTML
import ipywidgets as widgets

# ========== 1) CLUSTERING: Group selected countries by energy profile ==========
def cluster_selected_countries(df_selected, n_clusters=3):
    """
    Cluster selected countries based on their renewable energy mix.
    Returns: DataFrame with cluster assignments + visualization
    """
    if df_selected.empty or len(df_selected) < n_clusters:
        print("⚠️ Not enough countries selected for clustering (need at least", n_clusters, ")")
        return None, None
    
    # Extract latest year columns
    tech_cols = [c for c in df_selected.columns if '__F' in c]
    if not tech_cols:
        print("⚠️ No technology columns found")
        return None, None
    
    latest_year = max([int(c.split('__F')[1]) for c in tech_cols])
    latest_cols = [c for c in tech_cols if f'__F{latest_year}' in c]
    
    # Prepare data
    X = df_selected[['Country', 'Continent'] + latest_cols].copy()
    X_numeric = X[latest_cols].fillna(0)
    
    # Standardize
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_numeric)
    
    # K-means
    actual_clusters = min(n_clusters, len(df_selected))
    kmeans = KMeans(n_clusters=actual_clusters, random_state=42, n_init=10)
    X['Cluster'] = kmeans.fit_predict(X_scaled)
    
    # Statistics
    cluster_stats = X.groupby('Cluster')[latest_cols].mean()
    
    # ========== VISUALIZATION ==========
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Plot 1: Cluster distribution
    cluster_counts = X['Cluster'].value_counts().sort_index()
    colors = plt.cm.Set3(np.linspace(0, 1, actual_clusters))
    ax1.bar(cluster_counts.index, cluster_counts.values, color=colors)
    ax1.set_xlabel('Cluster ID', fontsize=12)
    ax1.set_ylabel('Number of Countries', fontsize=12)
    ax1.set_title('🔵 Cluster Distribution', fontsize=14, fontweight='bold')
    ax1.grid(axis='y', alpha=0.3)
    
    # Plot 2: Cluster profiles (heatmap)
    # Extract clean tech names
    tech_names = [c.split('__F')[0].replace('(MW)', '').replace('(GWh)', '').strip() 
                  for c in latest_cols]
    cluster_stats_renamed = cluster_stats.copy()
    cluster_stats_renamed.columns = tech_names
    
    sns.heatmap(cluster_stats_renamed, annot=True, fmt='.0f', cmap='YlOrRd', 
                ax=ax2, cbar_kws={'label': 'Average Capacity'})
    ax2.set_xlabel('Technology', fontsize=12)
    ax2.set_ylabel('Cluster ID', fontsize=12)
    ax2.set_title('🔥 Cluster Energy Profiles', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    # Print summary
    print(f"\n✅ Clustered {len(X)} countries into {actual_clusters} groups:")
    for cluster_id in range(actual_clusters):
        countries = X[X['Cluster'] == cluster_id]['Country'].tolist()
        print(f"  Cluster {cluster_id}: {', '.join(countries[:5])}" + 
              (f" (+{len(countries)-5} more)" if len(countries) > 5 else ""))
    
    return X[['Country', 'Continent', 'Cluster']], cluster_stats


# ========== 2) REGRESSION: Predict future trends for selected countries ==========
def predict_energy_trends(df_selected, tech_unit='Solar (MW)', top_n=5):
    """
    Linear regression to predict energy trends.
    Returns: predictions + visualization
    """
    if df_selected.empty:
        print("⚠️ No countries selected")
        return None
    
    # Find tech columns
    tech_cols = sorted([c for c in df_selected.columns if tech_unit in c and '__F' in c])
    if not tech_cols:
        print(f"⚠️ No data for {tech_unit}")
        return None
    
    # Extract years
    years = np.array([int(c.split('__F')[1]) for c in tech_cols])
    
    # Calculate total capacity per country (sum across all years)
    df_selected['Total'] = df_selected[tech_cols].sum(axis=1)
    top_countries = df_selected.nlargest(top_n, 'Total')['Country'].tolist()
    
    results = []
    
    # ========== VISUALIZATION ==========
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    colors = plt.cm.tab10(np.linspace(0, 1, len(top_countries)))
    
    for idx, country in enumerate(top_countries):
        country_data = df_selected[df_selected['Country'] == country]
        if country_data.empty:
            continue
        
        values = country_data[tech_cols].values.flatten()
        
        # Filter valid data
        mask = ~np.isnan(values) & (values > 0)
        if mask.sum() < 3:
            continue
        
        X = years[mask].reshape(-1, 1)
        y = values[mask]
        
        # Fit model
        model = LinearRegression()
        model.fit(X, y)
        
        # Predict future (5 years)
        future_years = np.arange(years.max() + 1, years.max() + 6).reshape(-1, 1)
        predictions = model.predict(future_years)
        
        r2 = r2_score(y, model.predict(X))
        
        results.append({
            'country': country,
            'slope': model.coef_[0],
            'r2': r2,
            'last_value': y[-1],
            'prediction_2025': predictions[0] if len(predictions) > 0 else None
        })
        
        # Plot 1: Historical + predictions
        ax1.plot(X.flatten(), y, 'o-', color=colors[idx], label=country, linewidth=2)
        ax1.plot(future_years.flatten(), predictions, '--', color=colors[idx], 
                 alpha=0.6, linewidth=2)
        
        # Plot 2: Growth rates
        ax2.barh(country, model.coef_[0], color=colors[idx])
    
    # Styling Plot 1
    ax1.set_xlabel('Year', fontsize=12)
    ax1.set_ylabel(f'{tech_unit}', fontsize=12)
    ax1.set_title('📈 Historical Trends + Predictions', fontsize=14, fontweight='bold')
    ax1.legend(loc='best', fontsize=9)
    ax1.grid(alpha=0.3)
    ax1.axvline(years.max(), color='red', linestyle=':', alpha=0.5, label='Today')
    
    # Styling Plot 2
    ax2.set_xlabel('Growth Rate (MW/year)', fontsize=12)
    ax2.set_title('⚡ Annual Growth Rates', fontsize=14, fontweight='bold')
    ax2.grid(axis='x', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print summary
    print(f"\nRegression Analysis for {tech_unit}:")
    for r in results:
        print(f"  {r['country']}: Growth={r['slope']:.1f} MW/yr, R²={r['r2']:.3f}, " +
              f"Predicted 2025={r['prediction_2025']:.0f} MW")
    
    return pd.DataFrame(results)


# ========== 3) ANOMALY DETECTION: Find unusual energy patterns ==========
def detect_energy_anomalies(df_selected, contamination=0.15):
    """
    Detect countries with unusual renewable energy patterns.
    Returns: anomaly labels + visualization
    """
    if df_selected.empty or len(df_selected) < 5:
        print("Need at least 5 countries for anomaly detection")
        return None
    
    # Extract latest year data
    tech_cols = [c for c in df_selected.columns if '__F' in c]
    latest_year = max([int(c.split('__F')[1]) for c in tech_cols])
    latest_cols = [c for c in tech_cols if f'__F{latest_year}' in c]
    
    X = df_selected[latest_cols].fillna(0)
    
    # Isolation Forest
    iso_forest = IsolationForest(contamination=contamination, random_state=42)
    predictions = iso_forest.fit_predict(X)
    
    df_anomalies = df_selected[['Country', 'Continent']].copy()
    df_anomalies['Anomaly'] = predictions
    df_anomalies['Anomaly_Label'] = df_anomalies['Anomaly'].map({1: 'Normal', -1: 'Anomaly'})
    
    anomalies = df_anomalies[df_anomalies['Anomaly'] == -1]
    
    # ========== VISUALIZATION ==========
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Plot 1: Anomaly distribution
    counts = df_anomalies['Anomaly_Label'].value_counts()
    colors_anom = ['#2ecc71', '#e74c3c']  # Green for normal, red for anomaly
    ax1.pie(counts.values, labels=counts.index, autopct='%1.1f%%', 
            colors=colors_anom, startangle=90, textprops={'fontsize': 12})
    ax1.set_title('🔍 Anomaly Detection Results', fontsize=14, fontweight='bold')
    
    # Plot 2: Anomaly scores
    scores = iso_forest.score_samples(X)
    countries_sorted = df_selected['Country'].tolist()
    colors_bars = ['#e74c3c' if p == -1 else '#3498db' for p in predictions]
    
    ax2.barh(countries_sorted, scores, color=colors_bars)
    ax2.set_xlabel('Anomaly Score (lower = more anomalous)', fontsize=12)
    ax2.set_title('📊 Anomaly Scores by Country', fontsize=14, fontweight='bold')
    ax2.grid(axis='x', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print summary
    print(f"\nDetected {len(anomalies)} anomalous countries:")
    if not anomalies.empty:
        for country in anomalies['Country'].tolist():
            print(f"  {country}")
    else:
        print("  (None detected)")
    
    return df_anomalies


# ========== 4) CORRELATION NETWORK: Find technology relationships ==========
def analyze_technology_correlations(df_selected, threshold=0.5):
    """
    Analyze correlations between energy technologies.
    Returns: correlation matrix + visualization
    """
    if df_selected.empty:
        print("No countries selected")
        return None
    
    # Extract latest year columns
    tech_cols = [c for c in df_selected.columns if '__F' in c]
    latest_year = max([int(c.split('__F')[1]) for c in tech_cols])
    latest_cols = [c for c in tech_cols if f'__F{latest_year}' in c]
    
    # Calculate correlation
    corr_matrix = df_selected[latest_cols].corr()
    
    # Extract clean tech names
    tech_names = [c.split('__F')[0].replace('(MW)', '').replace('(GWh)', '').strip() 
                  for c in latest_cols]
    corr_matrix.index = tech_names
    corr_matrix.columns = tech_names
    
    # ========== VISUALIZATION ==========
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
    
    # Plot 1: Correlation heatmap
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
    sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', 
                cmap='coolwarm', center=0, vmin=-1, vmax=1,
                ax=ax1, cbar_kws={'label': 'Correlation'})
    ax1.set_title('🔗 Technology Correlation Matrix', fontsize=14, fontweight='bold')
    
    # Plot 2: Strong correlations network
    strong_corrs = []
    for i in range(len(corr_matrix)):
        for j in range(i+1, len(corr_matrix)):
            corr = corr_matrix.iloc[i, j]
            if abs(corr) >= threshold:
                strong_corrs.append({
                    'Tech1': corr_matrix.index[i],
                    'Tech2': corr_matrix.columns[j],
                    'Correlation': corr
                })
    
    if strong_corrs:
        df_corr = pd.DataFrame(strong_corrs)
        colors_net = ['#e74c3c' if c < 0 else '#2ecc71' for c in df_corr['Correlation']]
        
        ax2.barh(range(len(df_corr)), df_corr['Correlation'].abs(), color=colors_net)
        ax2.set_yticks(range(len(df_corr)))
        ax2.set_yticklabels([f"{r['Tech1']} ↔ {r['Tech2']}" for _, r in df_corr.iterrows()])
        ax2.set_xlabel('|Correlation|', fontsize=12)
        ax2.set_title(f'Strong Correlations (|r| ≥ {threshold})', 
                      fontsize=14, fontweight='bold')
        ax2.grid(axis='x', alpha=0.3)
    else:
        ax2.text(0.5, 0.5, f'No strong correlations\n(threshold = {threshold})', 
                ha='center', va='center', fontsize=12, transform=ax2.transAxes)
        ax2.set_title('Strong Correlations', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    # Print summary
    print(f"\nCorrelation Analysis:")
    print(f"  Analyzed {len(tech_names)} technologies")
    print(f"  Found {len(strong_corrs)} strong correlations (|r| ≥ {threshold})")
    
    return corr_matrix


# ========== INTERACTIVE PANEL: Run all analyses ==========
def run_ml_analysis_on_selection():
    """
    Run all 4 ML techniques on currently selected countries.
    """
    global df_selected
    
    if df_selected.empty:
        print("No countries selected! Please select countries in the scatter plot above.")
        return
    
    print(f"\n{'='*60}")
    print(f"ML ANALYSIS ON {len(df_selected)} SELECTED COUNTRIES")
    print(f"{'='*60}\n")
    
    # 1) Clustering
    print("\n" + "="*60)
    print("1 CLUSTERING ANALYSIS")
    print("="*60)
    clusters, stats = cluster_selected_countries(df_selected, n_clusters=3)
    
    # 2) Regression
    print("\n" + "="*60)
    print("2 REGRESSION & FORECASTING")
    print("="*60)
    predictions = predict_energy_trends(df_selected, tech_unit='Solar (MW)', top_n=min(5, len(df_selected)))
    
    # 3) Anomaly Detection
    print("\n" + "="*60)
    print("3 ANOMALY DETECTION")
    print("="*60)
    anomalies = detect_energy_anomalies(df_selected, contamination=0.15)
    
    # 4) Correlation Analysis
    print("\n" + "="*60)
    print("4    CORRELATION NETWORK")
    print("="*60)
    correlations = analyze_technology_correlations(df_selected, threshold=0.5)
    
    print(f"\n{'='*60}")
    print("Analysis complete!")
    print(f"{'='*60}\n")

# ========== CREATE BUTTON TO RUN ANALYSIS ==========
button = widgets.Button(
    description='Run ML Analysis on Selection',
    button_style='success',
    tooltip='Click to analyze selected countries',
    icon='check',
    layout=widgets.Layout(width='300px', height='40px')
)

output = widgets.Output()

def on_button_click(b):
    with output:
        output.clear_output()
        run_ml_analysis_on_selection()

button.on_click(on_button_click)

display(widgets.VBox([
    widgets.HTML("<h3>Machine Learning Analysis Panel</h3>"),
    widgets.HTML("<p>Select countries in the scatter plot above, then click the button below:</p>"),
    button,
    output
]))

In [8]:
import pandas as pd
import numpy as np
import re

# === Carga
ev2 = pd.read_csv("Global_EV_clean.csv")
ev2["year"] = ev2["year"].astype(int)

# --- helper: pivotea (region, mode, year) a columnas <name>__FYYYY
def pivot_param2(data2, parameter2, powertrain2=None, name2="Val"):
    q2 = data2[data2["parameter"] == parameter2]
    if powertrain2 is not None:
        q2 = q2[q2["powertrain"] == powertrain2]
    q2 = q2[q2["mode"] != "EV"]  # quitamos el genérico
    if q2.empty:
        return pd.DataFrame(columns=["Country","mode"])
    g2 = q2.groupby(["region","mode","year"], as_index=False)["value"].sum()
    p2 = g2.pivot(index=["region","mode"], columns="year", values="value").sort_index(axis=1)
    p2.columns = [f"{name2}__F{int(y)}" for y in p2.columns]
    p2 = p2.reset_index().rename(columns={"region":"Country"})
    return p2

# --- ChargingStations: fast+slow por región-año replicado a cada mode != EV
cp2 = ev2[(ev2["parameter"] == "EV charging points") & (ev2["powertrain"].isin(["fast","slow"]))]

rgy2 = (cp2.groupby(["region","year"], as_index=False)["value"].sum()
          .rename(columns={"region":"Country","value":"ChargingStations"}))

base_modes2 = ev2.loc[ev2["mode"] != "EV", ["region","mode"]].drop_duplicates().rename(columns={"region":"Country"})

rep2 = base_modes2.merge(rgy2, on=["Country"], how="left")

cs2 = (rep2.pivot_table(index=["Country","mode"], columns="year", values="ChargingStations", aggfunc="first")
          .sort_index(axis=1).reset_index())
cs2.columns = [f"ChargingStations__F{int(c)}" if c not in ("Country","mode") else c for c in cs2.columns]

# --- Bloques métrica×tren
cfg2 = {
    "StockBEV":   ("EV stock",       "BEV"),
    "StockPHEV":  ("EV stock",       "PHEV"),
    "StockFCEV":  ("EV stock",       "FCEV"),
    "SalesBEV":   ("EV sales",       "BEV"),
    "SalesPHEV":  ("EV sales",       "PHEV"),
    "SalesFCEV":  ("EV sales",       "FCEV"),
    "StockShare": ("EV stock share", "EV"),
    # "SalesShare": ("EV sales share", "EV"),
}
blocks2 = [pivot_param2(ev2, p, pt, name2=k) for k,(p,pt) in cfg2.items()] + [cs2]

# --- Merge a ancho por país (una fila por Country×mode)
wide2 = base_modes2.copy()
for b2 in blocks2:
    if not b2.empty:
        wide2 = wide2.merge(b2, on=["Country","mode"], how="outer")

# --- Para energy_quad trabajaremos a nivel país: agregamos sobre modes
value_cols2 = [c for c in wide2.columns if re.search(r"__F\d{4}$", str(c))]
wide_ev2 = (wide2.groupby("Country", as_index=False)[value_cols2].sum(min_count=1))

# --- Años detectados
YEARS_ALL2 = sorted({int(m.group(1)) for c in value_cols2 for m in [re.search(r"__F(\d{4})$", c)] if m})
print(f"[OK2] wide_ev2 listo: {len(wide_ev2)} países | años {min(YEARS_ALL2)}–{max(YEARS_ALL2)} | cols={len(wide_ev2.columns)}")



[OK2] wide_ev2 listo: 49 países | años 2010–2023 | cols=113


In [11]:
from Isea import ParallelEnergy  # tu clase que usa energy_quad.js

# -------- parámetros de visual ----------
dims_all2 = (
    "StockBEV","StockPHEV","StockFCEV",
    "SalesBEV","SalesPHEV","SalesFCEV","ChargingStations",
    # "SalesShare",
)
YEARS2 = [y for y in YEARS_ALL2 if y >= 2015] or YEARS_ALL2
latest2 = max(YEARS2)
topN2 = 60  # más líneas → sube a gusto

# -------- top países por StockBEV en el año más reciente ----------
col_latest2 = f"StockBEV__F{latest2}"
wide_top2 = (wide_ev2.assign(**{col_latest2: pd.to_numeric(wide_ev2.get(col_latest2), errors="coerce").fillna(0.0)})
                        .sort_values(col_latest2, ascending=False)
                        .head(topN2)
                        .reset_index(drop=True))

# -------- builder de PACK para energy_quad.js ----------
def build_energy_pack2(wide_in2, dims2, years2, key_col2="Country"):
    records2 = []
    for _, row2 in wide_in2.iterrows():
        rec2 = {"label": row2[key_col2]}  # <- energy_quad busca r.label
        has_data2 = False
        for d2 in dims2:
            series2 = []
            for y2 in years2:
                v2 = row2.get(f"{d2}__F{y2}", 0.0)
                v2 = 0.0 if (pd.isna(v2) or not np.isfinite(v2)) else float(v2)
                if v2 != 0.0:
                    has_data2 = True
                series2.append(v2)
            rec2[d2] = series2
        if has_data2:
            records2.append(rec2)
    return {"years": [f"F{y}" for y in years2], "dims": list(dims2), "records": records2}

pack2 = build_energy_pack2(wide_top2, dims_all2, YEARS2)

# -------- Inicialización mínima del widget (dummy) ----------
dummy2 = pd.DataFrame({
    "Country": ["__init__"] * len(dims_all2),
    "Technology_std": list(dims_all2),
    f"F{latest2}": [0.0] * len(dims_all2),
})

w2 = ParallelEnergy(
    dummy2, [f"F{latest2}"],
    tech_col="Technology_std",   # distinto de label_col
    label_col="Country",
    dims=dims_all2,
    year_start=latest2,
    width=1280,
    reorder=True, normalize=False, log_axes=False
)

# -------- Opciones para energy_quad.js ----------
w2.options = {
    **w2.options,
    "title": f"EV landscape — {latest2}",
    "unit": "",
    "right_share": 0.36, "right_width": 520,
    "left_height": 540, "table_height": 230,
    "reorder": True, "normalize": False, "log_axes": False,
}

# Inyecta el pack y renderiza
w2.data = pack2
w2


ParallelEnergy(data={'years': ['F2015', 'F2016', 'F2017', 'F2018', 'F2019', 'F2020', 'F2021', 'F2022', 'F2023'…